# AutoGen Perusesimerkki

Tässä koodiesimerkissä käytät [AutoGen](https://aka.ms/ai-agents/autogen) AI-kehystä luodaksesi yksinkertaisen agentin.

Tämän esimerkin tavoitteena on näyttää vaiheet, joita käytämme myöhemmin lisäkoodiesimerkeissä, kun toteutamme erilaisia agenttimalleja.


## Tuo tarvittavat Python-paketit


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Luo asiakas

Tässä esimerkissä käytämme [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) -palvelua LLM:n käyttöön.

`model` on määritelty nimellä `gpt-4o-mini`. Kokeile vaihtaa mallia johonkin toiseen GitHub Models -markkinapaikalla saatavilla olevaan malliin nähdäksesi erilaisia tuloksia.

Nopeana testinä suoritetaan yksinkertainen kehotus - `Mikä on Ranskan pääkaupunki`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Agentin määrittäminen

Kun olemme nyt määrittäneet `client`-olion ja varmistaneet sen toimivuuden, voimme luoda `AssistantAgent`-agentin. Jokaiselle agentille voidaan määrittää:  
**nimi** - Lyhyt nimi, joka on hyödyllinen viitattaessa siihen monen agentin prosesseissa.  
**model_client** - Asiakas, jonka loit aiemmassa vaiheessa.  
**tools** - Käytettävissä olevat työkalut, joita agentti voi hyödyntää tehtävän suorittamisessa.  
**system_message** - Metakehotus, joka määrittää tehtävän, käyttäytymisen ja LLM:n sävyn.  

Voit muuttaa system message -viestiä nähdäksesi, miten LLM reagoi. Käsittelemme `tools`-työkaluja oppitunnilla #4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Suorita agentti

Alla oleva funktio suorittaa agentin. Käytämme `on_message`-metodia päivittämään agentin tilan uudella viestillä.

Tässä tapauksessa päivitämme tilan käyttäjältä tulevalla uudella viestillä, joka on `"Suunnittele minulle upea aurinkoinen loma"`.

Voit muuttaa viestin sisältöä nähdäksesi, miten LLM reagoi eri tavoin.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
